In [1]:
%cd ./bento

/home/bispl/PycharmProjects/SPMM_bento/bento


In [ ]:
from bentoml import HTTPServer
import numpy as np

# Initialize the server with the Bento
server = HTTPServer("schyena:latest", production=True, port=9999, host='0.0.0.0')

# Start the server (non-blocking by default)
server.start(blocking=False)

# Get a client to make requests to the server
client = server.get_client()

# Send a request using the client
result = client.celltype(np.array([[4.9, 3.0, 1.4, 0.2]]))
print(result)

In [2]:
import bentoml
import numpy as np
from bentoml.io import Text, NumpyNdarray, Multipart

spmm_model = bentoml.models.get("spmm:latest").to_runner()

svc = bentoml.Service(
    name="spmm", runners=[spmm_model]
)


@svc.api(input=Text(), output=NumpyNdarray())
async def SMILES_to_PV(smiles: str) -> np.ndarray:
    pv_numpy = await spmm_model.async_run(mode='s2p', s2p_smiles=smiles)
    return pv_numpy


@svc.api(input=Multipart(pv=NumpyNdarray(), mask=NumpyNdarray()), output=Text())
async def PV_to_SMILES(pv: np.ndarray, mask: np.ndarray) -> str:
    smiles_list = await spmm_model.async_run(mode='p2s', p2s_prop=pv, p2s_mask=mask, stochastic=True, k=2, n_sample=5)
    return ', '.join(smiles_list)


In [3]:
for runner in svc.runners:
    runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [4]:
# [INPUT] smiles: str
inp = {"smiles": 'c1ccccc1',}

result3 = await svc.apis["SMILES_to_PV"].func(**inp)
# [OUTPUT] result3: np.ndarray of shape(1, 53). The chemical property that each element correspond to is written in property_name.txt

SMILES-to-PV generation...
<class 'numpy.ndarray'>


In [5]:
result3

array([[ 1.3196881e+00, -2.0923682e+02,  1.3778400e+00,  4.0086021e+00,
         1.3090134e+00,  1.6224594e+00,  6.9206333e-01,  8.7461138e-01,
         8.8629150e-01, -1.7562079e+00, -3.1797457e-01, -1.1945078e+01,
        -4.8677278e-01, -3.7704479e+01,  8.1145874e+01,  7.1950597e-01,
         1.0214431e+00,  1.2122636e+00,  4.6828002e-02, -2.3598409e-01,
         1.8720291e+01,  7.7902802e+01,  2.5863609e+00,  1.0032511e-01,
        -9.7812366e-01,  1.2028702e+01,  5.5590191e+00,  3.3168030e+00,
        -3.5572699e-01, -6.5570199e-01, -3.1285763e-02,  8.3829498e+00,
         9.6435272e+01, -1.1060255e+00, -1.3402748e+00, -1.9088373e-01,
         4.9269855e-02,  4.2172730e-02, -5.4724681e-01,  5.6456578e-01,
         6.1490953e-01, -7.1379232e-01,  2.0000148e-01, -7.4872589e-01,
         6.3651544e-03, -1.2729344e+00, -1.2111440e-02,  1.4543086e-02,
         1.5330017e-02,  8.8671570e+00,  1.0430523e+00, -5.5257034e+00,
         3.5648382e-01]], dtype=float32)

In [6]:
from calc_property import calculate_property

p2i = dict()
with open('./property_name.txt', 'r') as f:
    lines = f.readlines()
for i, l in enumerate(lines):
    p2i[l.strip()] = i

# pv = calculate_property('COc1cccc(NC(=O)CN(C)C(=O)COC(=O)c2cc(c3cccs3)nc3ccccc23)c1').cpu().numpy()
# mask = np.zeros(53)

pv = np.zeros(53)
pv[p2i['ExactMolWt']] = 150.# ExactMolWt=property with the index 14
mask = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# [INPUT] pv: np.ndarray of shape (53,), mask: np.ndarray of shape (53,). The mask should only contain 0 and 1, where 1 means that property input will be ignored.
inp = {"pv":pv, "mask":mask}
result1 = await svc.apis["PV_to_SMILES"].func(**inp)
# [OUTPUT] result1: str. This string contains 5 SMILES output, concatenated with comma.

PV-to-SMILES generation...


100%|██████████| 5/5 [00:00<00:00,  7.93it/s]


In [7]:
result1

'N#CC1C(Cl)CCC1Cl, NC=C1CCCC(Cl)C1, CC(C)C1OC1C1CCCCC1, NC1C(Cl)CC(O)C1Cl, NC1CC2CCC(C1)C2Cl'